In [5]:
import pandas as pd
import random

import re
import pickle

In [20]:
### WORKING ON TWITTER DATA FOR SENTIMENTAL ANALYSIS

with open("Pickle Files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("Pickle Files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)

def getSentiment(text):

    # PREPROCESSING THE DATASET

    text = re.sub(r"^https://t.co/[A-Za-z0-9]*\s"," ",text)
    text = re.sub(r"\s+https://t.co/[A-Za-z0-9]*\s"," ",text)
    text = re.sub(r"\s+https://t.co/[A-Za-z0-9]*$"," ",text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]
            

In [42]:
dataset = pd.read_csv("sentiment-for-a-company-product-by-same-customer.csv")

In [43]:
# For Testing Purpose
customers = dataset.groupby("customer_id")
customer_list = dataset["customer_id"].unique()
customer_list

array([20422322, 40835037])

In [44]:
# users which are posting either all positive or negative reviews on different products of same brand

blocked_users = []
size = len(customer_list.tolist())
index = 0
for i in range(size):
    brand_df = customers.get_group(customer_list[i])
    
    brands = brand_df.groupby("product_parent")
    brands_list = brand_df["product_parent"].unique()
    
    no_of_brands = len(brands_list.tolist())
    for j in range(no_of_brands):
        
        product_df = brands.get_group(brands_list[j])
        no_of_products = len(product_df["product_id"])
        
        sentiment = getSentiment(product_df["review_body"][index])
                
        isSameSentiment = True
        if no_of_products==2:
            break
        for k in range(no_of_products):
            text = str(product_df["review_body"][index+k])
            if getSentiment(text)!=sentiment :
                isSameSentiment = False
                break;
        if(isSameSentiment):
            blocked_users.append(customer_list[i])
            break
    if(isSameSentiment):
        break;
    index += k

In [45]:
blocked_users

[20422322]

In [46]:
dataset.set_index("customer_id",inplace=True)

In [47]:
for customer in blocked_users:
    dataset.drop(customer,inplace=True)

In [48]:
dataset.reset_index()

,customer_id,marketplace,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
0,40835037,US,R8MEA6IGAHO0B,B00MC4CED8,137313254,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Something works perfectly, I'm ...",2015-08-31,207.156.210.106
1,40835037,US,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0,1,N,Y,five star,it's great,2015-08-31,193.93.167.87
2,40835037,US,R2Y0MM9YE6OP3P,B00QERR5CY,137313254,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0,0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,218.58.147.115
3,40835037,US,RRB9C05HDOD4O,B00QUFTPV4,137313254,abcGoodefgÂ® FBI Covert Acoustic Tube Earpiece...,Mobile_Electronics,4,0,0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31,218.86.71.213


In [13]:
#dataset.to_csv("review2.csv")